In [39]:
## Load libraries
# Interacting with the API
import json
from requests import get
from datetime import datetime
# Working with arrays, tables
import numpy as np
import pandas as pd
#from numba import jit
# Binary storage
import pyarrow as pa
import pyarrow.parquet as pq

def get_candles_gemini(url:str) -> pd.DataFrame:
    """Call Gemini Candles API"""
    candle_response = get(url)
    candle_report = pd.DataFrame(candle_response.json())
    candle_report.columns = ['time','open','high','low','close','volume']
    return candle_report

In [41]:

tradehistory_baseurl = "https://api.gemini.com/v2/"
tradehistory_endpoint = "candles/"

symbols = ["btcusd","ethbtc"]
symbols_url = ["".join([sym,"/"]) for sym in symbols]

interval = {
    "1m": "1m",
    "5m": "5m",
    "15m": "15m",
    "30m": "30m",
    "1hr": "1hr",
    "6hr": "6hr",
    "1day": "1day"
}

report_interal = interval['1day']


url_persymbol = ["".join([tradehistory_baseurl,tradehistory_endpoint,sym,report_interal]) for sym in symbols_url]

symbolurl_struct = {}
for symbol,url in zip(symbols,url_persymbol):
    symbolurl_struct[symbol] = url

# Set currency pair for report
report_symbol = symbols[1]


report_url = symbolurl_struct[report_symbol]
print(report_url)

https://api.gemini.com/v2/candles/ethbtc/1day


In [42]:
candle_report = get_candles_gemini(report_url)

print(candle_report.head())
print("\n")
print(candle_report.shape)

            time     open     high      low    close       volume
0  1616040000000  0.03124  0.03124  0.03049  0.03096   977.991021
1  1615953600000  0.03183  0.03244  0.03079  0.03124  3007.687423
2  1615867200000  0.03225  0.03271  0.03150  0.03183  1484.684485
3  1615780800000  0.03120  0.03231  0.03076  0.03225  3063.958026
4  1615694400000  0.03119  0.03154  0.03089  0.03120  1280.342195


(366, 6)


In [43]:
time_col = candle_report['time']
print(time_col[0])

test_conv = time_col[0]


1616040000000


In [44]:
print(test_conv)

1616040000000


In [45]:
def tsms_convert(time_ms):
    datetimed = pd.to_datetime(time_ms, infer_datetime_format=True, unit='ms')
    return datetimed

def apply_timeconv(time_coln,func):
    convertedtime = pd.Series(map(func,time_coln))
    return convertedtime

time_col = apply_timeconv(time_col,tsms_convert)

csv_rep = candle_report.copy()

csv_rep['time'] = time_col

print(csv_rep[:20])

                  time     open     high      low    close       volume
0  2021-03-18 04:00:00  0.03124  0.03124  0.03049  0.03096   977.991021
1  2021-03-17 04:00:00  0.03183  0.03244  0.03079  0.03124  3007.687423
2  2021-03-16 04:00:00  0.03225  0.03271  0.03150  0.03183  1484.684485
3  2021-03-15 04:00:00  0.03120  0.03231  0.03076  0.03225  3063.958026
4  2021-03-14 04:00:00  0.03119  0.03154  0.03089  0.03120  1280.342195
5  2021-03-13 04:00:00  0.03105  0.03185  0.03066  0.03119  4016.523557
6  2021-03-12 04:00:00  0.03185  0.03190  0.03049  0.03105  1618.797415
7  2021-03-11 04:00:00  0.03220  0.03220  0.03147  0.03185  1092.378860
8  2021-03-10 04:00:00  0.03308  0.03368  0.03176  0.03220  5474.115299
9  2021-03-09 04:00:00  0.03443  0.03443  0.03305  0.03308  2583.031787
10 2021-03-08 04:00:00  0.03409  0.03520  0.03376  0.03443  1864.197317
11 2021-03-07 04:00:00  0.03329  0.03416  0.03243  0.03409  1544.696111
12 2021-03-06 04:00:00  0.03148  0.03433  0.03148  0.03329  4777

In [47]:
# Calculate simple returns 
def calculate_return_series(series: pd.Series) -> pd.Series:
    """
    Calculates the return series of a time series.
    The first value will always be NaN.
    Output series retains the index of the input series.
    """
    shifted_series = series.shift(1, axis=0)
    return series / shifted_series - 1


def vwap(vola, higha, lowa) -> pd.Series:
    new_arr = np.cumsum(vola*(higha+lowa)/2) / np.cumsum(vola)
    return pd.Series(new_arr)

In [48]:
close_series = pd.Series(csv_rep["close"])

csv_rep['simple_rtn'] = calculate_return_series(close_series)

In [49]:

volume_arr = csv_rep.volume.values
high_arr = csv_rep.high.values
low_arr = csv_rep.low.values



csv_rep['vwap'] = vwap(volume_arr, high_arr, low_arr)

csv_rep.head()

,time,open,high,low,close,volume,simple_rtn,vwap
0,2021-03-18 04:00:00,0.03124,0.03124,0.03049,0.03096,977.991021,NaN,0.030865
1,2021-03-17 04:00:00,0.03183,0.03244,0.03079,0.03124,3007.687423,0.009044,0.031431
2,2021-03-16 04:00:00,0.03225,0.03271,0.03150,0.03183,1484.684485,0.018886,0.031614
3,2021-03-15 04:00:00,0.03120,0.03231,0.03076,0.03225,3063.958026,0.013195,0.031586
4,2021-03-14 04:00:00,0.03119,0.03154,0.03089,0.03120,1280.342195,-0.032558,0.031537


In [50]:
# Save csv
csv_rep.to_csv('OHLC_daily_ethbtc_gemini_210318.csv', index=False)

In [ ]:
#parquet.read_table("./tradehistory_daily_btcusd_gemini_210317.parquet", columns=['time'])

### Calculate VWAP

In [29]:
candle_report.head()



,time,open,high,low,close,volume
0,1616040000000,58687.36,60099.99,57493.67,58468.14,1011.498722
1,1615953600000,55548.73,59567.59,54144.95,58687.36,1950.943235
2,1615867200000,54255.04,57185.78,53589.00,55548.73,1350.969382
3,1615780800000,60362.18,60561.59,53238.69,54255.04,3453.090624
4,1615694400000,61354.75,61500.82,58750.00,60362.18,1286.146361


In [46]:
# Calculate simple returns 
def calculate_return_series(series: pd.Series) -> pd.Series:
    """
    Calculates the return series of a time series.
    The first value will always be NaN.
    Output series retains the index of the input series.
    """
    shifted_series = series.shift(1, axis=0)
    return series / shifted_series - 1


def vwap(vola, higha, lowa) -> pd.Series:
    new_arr = np.cumsum(vola*(higha+lowa)/2) / np.cumsum(vola)
    return pd.Series(new_arr)

In [51]:

close_series = pd.Series(candle_report["close"])

candle_report['simple_rtn'] = calculate_return_series(close_series)



In [52]:
candle_report.head()

,time,open,high,low,close,volume,simple_rtn
0,1616040000000,0.03124,0.03124,0.03049,0.03096,977.991021,NaN
1,1615953600000,0.03183,0.03244,0.03079,0.03124,3007.687423,0.009044
2,1615867200000,0.03225,0.03271,0.03150,0.03183,1484.684485,0.018886
3,1615780800000,0.03120,0.03231,0.03076,0.03225,3063.958026,0.013195
4,1615694400000,0.03119,0.03154,0.03089,0.03120,1280.342195,-0.032558


In [53]:


volume_arr = candle_report.volume.values
high_arr = candle_report.high.values
low_arr = candle_report.low.values



candle_report['vwap'] = vwap(volume_arr, high_arr, low_arr)

candle_report.head()


,time,open,high,low,close,volume,simple_rtn,vwap
0,1616040000000,0.03124,0.03124,0.03049,0.03096,977.991021,NaN,0.030865
1,1615953600000,0.03183,0.03244,0.03079,0.03124,3007.687423,0.009044,0.031431
2,1615867200000,0.03225,0.03271,0.03150,0.03183,1484.684485,0.018886,0.031614
3,1615780800000,0.03120,0.03231,0.03076,0.03225,3063.958026,0.013195,0.031586
4,1615694400000,0.03119,0.03154,0.03089,0.03120,1280.342195,-0.032558,0.031537


In [54]:
pqtable = pa.Table.from_pandas(candle_report)
pq.write_table(pqtable, 'OHLC_daily_ethbtc_gemini_210318.parquet')

## Gemini Ticker V2 API

Link: [ticker v2 api](https://docs.gemini.com/rest-api/#ticker)

Description: This endpoint retrieves information about recent trading activity for the provided symbol.

HTTP REQUEST
- GET https://api.gemini.com/v2/ticker/:symbol

URL PARAMETERS: None

SANDBOX: api.sandbox.gemini.com/v2 

RESPONSE

The response will be an object containing symbol, open, high, low, close, changes, bid, ask


### Set Libraries

In [2]:

from requests import get

import numpy as np
import pandas as pd

import pyarrow as pa
import pyarrow.parquet as parquet

In [5]:

tradehistory_baseurl = "https://api.gemini.com/v2/"
tradehistory_endpoint = "ticker/"

symbols = ["btcusd","ethbtc"]

url_persymbol = ["".join([tradehistory_baseurl,tradehistory_endpoint,sym]) for sym in symbols]

symbolurl_struct = {}
for symbol,url in zip(symbols,url_persymbol):
    symbolurl_struct[symbol] = url

# Set currency pair for report
report_symbol = symbols[0]

report_url = symbolurl_struct[report_symbol]
print(report_url)

https://api.gemini.com/v2/ticker/btcusd


In [6]:
tickerv2resp = get(report_url)

In [10]:
pd.DataFrame(tickerv2resp.json())

,symbol,open,high,low,close,changes,bid,ask
0,BTCUSD,55546.12,60099.99,55450.01,59373.32,59105.61,59353.06,59366.62
1,BTCUSD,55546.12,60099.99,55450.01,59373.32,59546.27,59353.06,59366.62
2,BTCUSD,55546.12,60099.99,55450.01,59373.32,57886.65,59353.06,59366.62
3,BTCUSD,55546.12,60099.99,55450.01,59373.32,57841.45,59353.06,59366.62
4,BTCUSD,55546.12,60099.99,55450.01,59373.32,58086.26,59353.06,59366.62
5,BTCUSD,55546.12,60099.99,55450.01,59373.32,58177.43,59353.06,59366.62
6,BTCUSD,55546.12,60099.99,55450.01,59373.32,58494.23,59353.06,59366.62
7,BTCUSD,55546.12,60099.99,55450.01,59373.32,58470.87,59353.06,59366.62
8,BTCUSD,55546.12,60099.99,55450.01,59373.32,58005.61,59353.06,59366.62
9,BTCUSD,55546.12,60099.99,55450.01,59373.32,58379.77,59353.06,59366.62
